In [7]:
import torch.nn as nn
class NeighborhoodAttention(nn.Module): #It can only use static size as input,but you can define a new input size if you wish.
    def __init__(self,input_size, dim, num_heads,window_size=7, qkv_bias=True, qk_scale=None, attn_drop=0., proj_drop=0.):
        super().__init__()
        assert window_size%2 == 1,'windowsize must be odd.'
        self.dim = dim
        self.window_size = window_size
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = qk_scale or head_dim ** -0.5
        self.qkv = nn.Conv2d(dim,dim*3,1, bias=qkv_bias)
        self.proj = nn.Conv2d(dim, dim, 1)
        self.proj_drop = nn.Dropout(proj_drop)
        self.attn_drop = nn.Dropout(attn_drop)
        self.pad_idx = nn.ReplicationPad2d(self.window_size//2)
        self.relative_bias = nn.Parameter(torch.zeros((2*self.window_size-1)**2,num_heads))
        trunc_normal_(self.relative_bias, std=.02)
        self.idx_h = torch.arange(0,window_size)
        self.idx_w = torch.arange(0,window_size)
        self.idx_k = ((self.idx_h.unsqueeze(-1) * (2*self.window_size-1)) + self.idx_w).view(-1)
        self.set_input_size(input_size)

    def forward(self, x):
        x = self.attention(x)
        x = self.proj(x)
        x = self.proj_drop(x)
        return x

    def attention(self,x):
        B,C,H,W = x.shape
        assert H >= self.window_size and W >= self.window_size,'input size must not be smaller than window size'
        qkv = self.qkv(x).view(B, 3,self.num_heads,C//self.num_heads,H*W).permute(1, 0, 2, 4, 3)
        q, k, v = qkv[0], qkv[1], qkv[2]
        q = q * self.scale
        attn = q.unsqueeze(3) @ k[:,:,self.attn_idx].unsqueeze(2) #B,nh,L,1,K^2
        attn = attn + self.relative_bias[self.bias_idx].permute(2, 0, 1).unsqueeze(2)
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)
        x = (attn @ v[:,:,self.attn_idx]).squeeze(3).transpose(-1,-2).contiguous().view(B,C,H,W)
        return x
        
    def get_bias_idx(self,H,W):
        num_repeat_h = torch.ones(self.window_size,dtype=torch.long)
        num_repeat_w = torch.ones(self.window_size,dtype=torch.long)
        num_repeat_h[self.window_size//2] = H-(self.window_size-1)
        num_repeat_w[self.window_size//2] = W-(self.window_size-1)
        bias_hw = (self.idx_h.repeat_interleave(num_repeat_h).unsqueeze(-1) * (2*self.window_size-1)) + self.idx_w.repeat_interleave(num_repeat_w)
        bias_idx = bias_hw.unsqueeze(-1) + self.idx_k
        return bias_idx.view(-1,self.window_size**2)

      
    def get_attn_idx(self,H,W):
        H_ = H - (self.window_size - 1)
        W_ = W - (self.window_size - 1)
        attn_idx = torch.arange(0,H_*W_,dtype=torch.float).view(1,1,H_,W_)
        attn_idx = self.pad_idx(attn_idx).view(-1).type(torch.long)
        attn_idx = self.get_unfold_idx(H,W)[attn_idx]
        return attn_idx
    
    def get_unfold_idx(self,H,W):
        H_ = H-(self.window_size-1)
        W_ = W-(self.window_size-1)
        h_idx = torch.arange(W_).repeat(H_)
        w_idx = torch.arange(H_).repeat_interleave(W_) * W
        k_idx_1 = torch.arange(self.window_size).repeat(self.window_size)
        k_idx_2 = torch.arange(self.window_size).repeat_interleave(self.window_size) * W
        k_idx = k_idx_1 + k_idx_2
        hw_idx = h_idx + w_idx
        unfold_idx = hw_idx[:,None] + k_idx
        return unfold_idx
    def set_input_size(self,input_size):
        H,W = input_size
        self.H,self.W = H,W
        assert H >= self.window_size and W >= self.window_size,'input size must not be smaller than window size'
        attn_idx = self.get_attn_idx(H,W)
        bias_idx = self.get_bias_idx(H,W)
        self.register_buffer("attn_idx", attn_idx)
        self.register_buffer("bias_idx",bias_idx)
        

In [8]:
import torch
from torch import nn
from torch.nn import functional as F
from timm.models.layers import DropPath, trunc_normal_

class Channel_Layernorm(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.ln = nn.LayerNorm(dim)
    def forward(self, x):
        x = x.permute(0, 2, 3, 1)
        x = self.ln(x)
        x = x.permute(0, 3, 1, 2)
        return x
    
class Mlp(nn.Module):
    def __init__(self, in_features, hidden_features=None, out_features=None, act_layer=nn.GELU, drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = nn.Conv2d(in_features, hidden_features,1)
        self.act = act_layer()
        self.fc2 = nn.Conv2d(hidden_features, out_features,1)
        self.drop = nn.Dropout(drop)
    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x

In [9]:
class NATLayer(nn.Module):
    def __init__(self,input_size, dim, num_heads,window_size=7,
                 mlp_ratio=4., qkv_bias=True, qk_scale=None, drop=0., attn_drop=0., drop_path=0.,
                 act_layer=nn.GELU, norm_layer=Channel_Layernorm, layer_scale=None):
        super().__init__()
        self.dim = dim
        self.num_heads = num_heads
        self.mlp_ratio = mlp_ratio
        self.norm1 = norm_layer(dim)
        self.attn = NeighborhoodAttention(input_size, dim, num_heads,window_size,qkv_bias, qk_scale, attn_drop, drop)
        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()
        self.norm2 = norm_layer(dim)
        self.mlp = Mlp(in_features=dim, hidden_features=int(dim * mlp_ratio), act_layer=act_layer, drop=drop)
        
    def forward(self, x):
        shortcut = x
        x = self.norm1(x)
        x = self.attn(x)
        x = shortcut + self.drop_path(x)
        x = x + self.drop_path(self.mlp(self.norm2(x)))
        return x
    
    def set_input_size(self,input_size):
        self.attn.set_input_size(input_size)

In [10]:
model = NATLayer((28,28),128,4)
img = torch.rand(2,128,56,56)
model(img)

RuntimeError: The size of tensor a (3136) must match the size of tensor b (784) at non-singleton dimension 3

In [4]:
def test():
    print('it is cpu')
    model = NATLayer((28,28),128,4)
    img = torch.rand(2,128,56,56)
    try:
        print(model(img).shape)
    except:
        print('error')
        model.set_input_size((56,56))
        print(model(img).shape)
    print('cpu_success\n')

In [5]:
test()

it is cpu
error
torch.Size([2, 128, 56, 56])
cpu_success



In [6]:
def test_cuda():
    print('it is cuda')
    model = NATLayer((28,28),128,4).cuda()
    img = torch.rand(2,128,56,56).cuda()
    try:
        print(model(img).shape)
    except:
        print('error')
        model.set_input_size((56,56))
        print(model(img).shape)
    print('success')
    print('cuda_success\n')

In [7]:
test_cuda()

it is cuda
error
torch.Size([2, 128, 56, 56])
success
cuda_success



In [2]:
import torch
import torch.nn as nn

class NeighborhoodAttention(nn.Module):
    def __init__(self, input_size, dim, num_heads, window_size=7, qkv_bias=True, qk_scale=None, attn_drop=0., proj_drop=0.):
        super().__init__()
        assert window_size % 2 == 1, 'Window size must be odd.'
        
        self.dim = dim
        self.window_size = window_size
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = head_dim ** -0.5
        
        self.qkv = nn.Conv2d(dim, dim * 3, 1, bias=True)
        self.proj = nn.Conv2d(dim, dim, 1)
        self.proj_drop = nn.Dropout(proj_drop)
        self.attn_drop = nn.Dropout(attn_drop)
        self.pad_idx = nn.ReplicationPad2d(window_size // 2)
        
        self.relative_bias = nn.Parameter(torch.zeros((2 * window_size - 1) ** 2, num_heads))
        nn.init.trunc_normal_(self.relative_bias, std=0.02)
        
        self.set_input_size(input_size)
    
    def forward(self, x):
        x = self.attention(x)
        x = self.proj(x)
        return self.proj_drop(x)
    
    def attention(self, x):
        B, C, H, W = x.shape
        assert H >= self.window_size and W >= self.window_size, 'Input size must be at least window size'
        
        qkv = self.qkv(x).view(B, 3, self.num_heads, C // self.num_heads, H * W).permute(1, 0, 2, 4, 3)
        q, k, v = qkv[0] * self.scale, qkv[1], qkv[2]
        
        attn = (q @ k[:, :, self.attn_idx].transpose(-1, -2))
        attn += self.relative_bias[self.bias_idx].permute(2, 0, 1).unsqueeze(2)
        attn = self.attn_drop(attn.softmax(dim=-1))
        
        return (attn @ v[:, :, self.attn_idx]).squeeze(3).transpose(-1, -2).contiguous().view(B, C, H, W)
    
    def set_input_size(self, input_size):
        H, W = input_size
        self.register_buffer("attn_idx", self.get_attn_idx(H, W))
        self.register_buffer("bias_idx", self.get_bias_idx(H, W))
    
    def get_attn_idx(self, H, W):
        attn_idx = torch.arange(0, (H - self.window_size + 1) * (W - self.window_size + 1)).view(1, 1, H - self.window_size + 1, W - self.window_size + 1)
        return self.pad_idx(attn_idx.float()).view(-1).long()
    
    def get_bias_idx(self, H, W):
        return torch.arange(self.window_size ** 2).repeat(H * W, 1)


In [29]:
window_size =7
H,W = 14,14

In [49]:
H_ = H-(window_size-1)
W_ = W-(window_size-1)
h_idx = torch.arange(W_).repeat(H_)
w_idx = torch.arange(H_).repeat_interleave(W_) * W
hw_idx = h_idx + w_idx
(h_idx, w_idx, hw_idx)
# print(len(h_idx), len(w_idx), len(hw_idx))

(tensor([0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7,
         0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7,
         0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7]),
 tensor([ 0,  0,  0,  0,  0,  0,  0,  0, 14, 14, 14, 14, 14, 14, 14, 14, 28, 28,
         28, 28, 28, 28, 28, 28, 42, 42, 42, 42, 42, 42, 42, 42, 56, 56, 56, 56,
         56, 56, 56, 56, 70, 70, 70, 70, 70, 70, 70, 70, 84, 84, 84, 84, 84, 84,
         84, 84, 98, 98, 98, 98, 98, 98, 98, 98]),
 tensor([  0,   1,   2,   3,   4,   5,   6,   7,  14,  15,  16,  17,  18,  19,
          20,  21,  28,  29,  30,  31,  32,  33,  34,  35,  42,  43,  44,  45,
          46,  47,  48,  49,  56,  57,  58,  59,  60,  61,  62,  63,  70,  71,
          72,  73,  74,  75,  76,  77,  84,  85,  86,  87,  88,  89,  90,  91,
          98,  99, 100, 101, 102, 103, 104, 105]))

In [50]:
k_idx_1 = torch.arange(window_size).repeat(window_size)
k_idx_2 = torch.arange(window_size).repeat_interleave(window_size) * W
k_idx = k_idx_1 + k_idx_2
(k_idx_1, k_idx_2, k_idx)
# print(len(k_idx_1), len(k_idx_2), len(k_idx))

(tensor([0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2,
         3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5,
         6]),
 tensor([ 0,  0,  0,  0,  0,  0,  0, 14, 14, 14, 14, 14, 14, 14, 28, 28, 28, 28,
         28, 28, 28, 42, 42, 42, 42, 42, 42, 42, 56, 56, 56, 56, 56, 56, 56, 70,
         70, 70, 70, 70, 70, 70, 84, 84, 84, 84, 84, 84, 84]),
 tensor([ 0,  1,  2,  3,  4,  5,  6, 14, 15, 16, 17, 18, 19, 20, 28, 29, 30, 31,
         32, 33, 34, 42, 43, 44, 45, 46, 47, 48, 56, 57, 58, 59, 60, 61, 62, 70,
         71, 72, 73, 74, 75, 76, 84, 85, 86, 87, 88, 89, 90]))

In [57]:
unfold_idx = hw_idx.unsqueeze(1) + k_idx #hw_idx[:,None] 
unfold_idx

tensor([[  0,   1,   2,  ...,  88,  89,  90],
        [  1,   2,   3,  ...,  89,  90,  91],
        [  2,   3,   4,  ...,  90,  91,  92],
        ...,
        [103, 104, 105,  ..., 191, 192, 193],
        [104, 105, 106,  ..., 192, 193, 194],
        [105, 106, 107,  ..., 193, 194, 195]])

In [62]:
attn_idx = torch.arange(0,H_*W_).float().view(1,1,H_,W_)
attn_idx


tensor([[[[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11., 12., 13., 14., 15.],
          [16., 17., 18., 19., 20., 21., 22., 23.],
          [24., 25., 26., 27., 28., 29., 30., 31.],
          [32., 33., 34., 35., 36., 37., 38., 39.],
          [40., 41., 42., 43., 44., 45., 46., 47.],
          [48., 49., 50., 51., 52., 53., 54., 55.],
          [56., 57., 58., 59., 60., 61., 62., 63.]]]])

In [65]:
attn_idx.shape

torch.Size([1, 1, 8, 8])

In [64]:
pad_idx = nn.ReplicationPad2d(window_size//2)
pad_idx

ReplicationPad2d((3, 3, 3, 3))

In [71]:
attn_idx = pad_idx(attn_idx).view(-1).type(torch.long)
attn_idx

tensor([ 0,  0,  0,  0,  1,  2,  3,  4,  5,  6,  7,  7,  7,  7,  0,  0,  0,  0,
         1,  2,  3,  4,  5,  6,  7,  7,  7,  7,  0,  0,  0,  0,  1,  2,  3,  4,
         5,  6,  7,  7,  7,  7,  0,  0,  0,  0,  1,  2,  3,  4,  5,  6,  7,  7,
         7,  7,  8,  8,  8,  8,  9, 10, 11, 12, 13, 14, 15, 15, 15, 15, 16, 16,
        16, 16, 17, 18, 19, 20, 21, 22, 23, 23, 23, 23, 24, 24, 24, 24, 25, 26,
        27, 28, 29, 30, 31, 31, 31, 31, 32, 32, 32, 32, 33, 34, 35, 36, 37, 38,
        39, 39, 39, 39, 40, 40, 40, 40, 41, 42, 43, 44, 45, 46, 47, 47, 47, 47,
        48, 48, 48, 48, 49, 50, 51, 52, 53, 54, 55, 55, 55, 55, 56, 56, 56, 56,
        57, 58, 59, 60, 61, 62, 63, 63, 63, 63, 56, 56, 56, 56, 57, 58, 59, 60,
        61, 62, 63, 63, 63, 63, 56, 56, 56, 56, 57, 58, 59, 60, 61, 62, 63, 63,
        63, 63, 56, 56, 56, 56, 57, 58, 59, 60, 61, 62, 63, 63, 63, 63])

In [74]:
attn_idx  = unfold_idx[attn_idx]
attn_idx

tensor([[  0,   1,   2,  ...,  88,  89,  90],
        [  0,   1,   2,  ...,  88,  89,  90],
        [  0,   1,   2,  ...,  88,  89,  90],
        ...,
        [105, 106, 107,  ..., 193, 194, 195],
        [105, 106, 107,  ..., 193, 194, 195],
        [105, 106, 107,  ..., 193, 194, 195]])

In [79]:
bias = torch.arange(window_size ** 2).repeat(H * W, 1)
bias

tensor([[ 0,  1,  2,  ..., 46, 47, 48],
        [ 0,  1,  2,  ..., 46, 47, 48],
        [ 0,  1,  2,  ..., 46, 47, 48],
        ...,
        [ 0,  1,  2,  ..., 46, 47, 48],
        [ 0,  1,  2,  ..., 46, 47, 48],
        [ 0,  1,  2,  ..., 46, 47, 48]])

In [80]:
attn_idx.shape

torch.Size([196, 49])